In [17]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 50)
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import boxcox
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score, silhouette_score
import precision_recall_cutoff

# Reading the csv file
train = pd.read_csv('train_dataset.csv')

# Defining the file to be read from s3 bucket
test = pd.read_csv('test_dataset.csv')
test = test.dropna()
test.shape

(498121, 44)

In [18]:
train.shape

(1879, 45)

In [19]:
train.head()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud,interaction_1,interaction_2,interaction_3,interaction_4,average_seconds_per_item,trustLevel_1,trustLevel_2,trustLevel_3,trustLevel_4,trustLevel_5,trustLevel_6,totalScanTimeInSeconds_0_1,grandTotal_0_1,lineItemVoids_0_1,scansWithoutRegistration_0_1,quantityModifications_0_1,scannedLineItemsPerSecond_0_1,valuePerSecond_0_1,lineItemVoidsPerPosition_0_1,average_seconds_per_item_01,Labels,Labels_0,Labels_1,Labels_2,Labels_3,Labels_4,Labels_5,interaction_5,interaction_6,interaction_7,interaction_8,heredity_1,heredity_2,heredity_3,interaction_9
0,5,1054,54.70,7,0,3,0.027514,0.051898,0.241379,0,0,0,0,1,19.27,0,0,0,0,1,0,0.575178,0.547174,0.636364,0.0,0.6,0.004045,0.001370,0.021944,0.000131,0,1,0,0,0,0,0,0.000000,0.002427,0.314722,0,0,0,0,0
1,3,108,27.36,5,2,4,0.129630,0.253333,0.357143,0,0,0,0,1,3.95,0,0,1,0,0,0,0.057955,0.273637,0.454545,0.2,0.8,0.019364,0.006689,0.032468,0.000027,1,0,1,0,0,0,0,0.090909,0.015491,0.015859,0,0,0,3,0
2,3,1516,62.16,3,10,5,0.008575,0.041003,0.230769,0,0,0,0,1,24.39,0,0,1,0,0,0,0.827775,0.621811,0.272727,1.0,1.0,0.001204,0.001083,0.020979,0.000165,5,0,0,0,0,0,1,0.272727,0.001204,0.514719,0,0,0,15,0
3,6,1791,92.31,8,4,4,0.016192,0.051541,0.275862,0,0,0,0,1,19.40,0,0,0,0,0,1,0.978130,0.923462,0.727273,0.4,0.8,0.002347,0.001361,0.025078,0.000132,2,0,0,1,0,0,0,0.290909,0.001877,0.903266,0,0,0,12,0
4,5,430,81.53,3,7,2,0.062791,0.189605,0.111111,0,0,0,0,1,5.27,0,0,0,0,1,0,0.234008,0.815608,0.272727,0.7,0.4,0.009337,0.005007,0.010101,0.000036,4,0,0,0,0,1,0,0.190909,0.003735,0.190858,0,0,0,20,0


In [20]:
X = train[['totalScanTimeInSeconds', 'interaction_1', 'interaction_4', 'heredity_1', 'interaction_9', 'heredity_2', 'interaction_5']]
Y = train['fraud']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

X_train_7 = X_train
X_train_6 = X_train[['totalScanTimeInSeconds', 'interaction_1', 'interaction_4', 'heredity_1', 'interaction_9', 'heredity_2']]
X_train_5 = X_train [['totalScanTimeInSeconds', 'interaction_1', 'interaction_4', 'heredity_1', 'interaction_9']]

### Fitting the model with the top 5

In [ ]:
## Defining the hyper-parameters for RF
RF_param_grid = {'n_estimators': [100, 300, 500],
                 'min_samples_split': [10, 15], 
                 'min_samples_leaf': [5, 7], 
                 'max_depth' : [3, 5, 7]}

# Defining customized scoring function
my_score_function = make_scorer(cost_function, greater_is_better = True, needs_proba = True)

# Performing GridSearch
RF_grid_search = GridSearchCV(RandomForestClassifier(), RF_param_grid, cv = 3, scoring = my_score_function, n_jobs = -1).fit(X_train_5, Y_train)

# Extracting the best model
RF_model = RF_grid_search.best_estimator_
RF_model

### Fitting the model with the top 6

In [ ]:
## Defining the hyper-parameters for RF
RF_param_grid = {'n_estimators': [100, 300, 500],
                 'min_samples_split': [10, 15], 
                 'min_samples_leaf': [5, 7], 
                 'max_depth' : [3, 5, 7]}

# Defining customized scoring function
my_score_function = make_scorer(cost_function, greater_is_better = True, needs_proba = True)

# Performing GridSearch
RF_grid_search = GridSearchCV(RandomForestClassifier(), RF_param_grid, cv = 3, scoring = my_score_function, n_jobs = -1).fit(X_train, Y_train)

# Extracting the best model
RF_model = RF_grid_search.best_estimator_
RF_model

### Fitting the model with the top 7

In [ ]:
## Defining the hyper-parameters for RF
RF_param_grid = {'n_estimators': [100, 300, 500],
                 'min_samples_split': [10, 15], 
                 'min_samples_leaf': [5, 7], 
                 'max_depth' : [3, 5, 7]}

# Defining customized scoring function
my_score_function = make_scorer(cost_function, greater_is_better = True, needs_proba = True)

# Performing GridSearch
RF_grid_search = GridSearchCV(RandomForestClassifier(), RF_param_grid, cv = 3, scoring = my_score_function, n_jobs = -1).fit(X_train, Y_train)

# Extracting the best model
RF_model = RF_grid_search.best_estimator_
RF_model